In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/miramedix')

In [ ]:
import json

with open('ner_annotation.json', 'r') as f:
  annot_data = json.load(f)

In [ ]:
data = []
for i in range(len(annot_data)):
  i_data = [annot_data[i]['id'], annot_data[i]['annotations'], annot_data[i]['data']]
  data.append(i_data)

In [ ]:
data_sorted = sorted(data, key=lambda tup: tup[0])

In [ ]:
import json

with open('descriptions.json', 'r') as f:
  texts = json.load(f)

In [ ]:
import re

ner_data = {}
for text_id in texts.keys():
  clean_text = texts[text_id].strip()
  #sents = [s + '.' for s in clean_text.split('. ') if s]
  for_text = []
  for sent_id in range(len(data_sorted)):
    sent = data_sorted[sent_id][2]['text']
    sent_clean = re.sub(' / ', '/', sent)
    if sent_clean in clean_text:
      for_text.append(data_sorted[sent_id])
  ner_data[text_id] = for_text

In [ ]:
texts['1']

'\nPatient is a 45-year-old man with a history of anaplastic astrocytoma of the spine complicated by severe lower extremity weakness and urinary retention s/p Foley catheter, high-dose steroids, hypertension, and chronic pain. The tumor is located in the T-L spine, unresectable anaplastic astrocytoma s/p radiation. Complicated by progressive lower extremity weakness and urinary retention. Patient initially presented with RLE weakness where his right knee gave out with difficulty walking and right anterior thigh numbness. MRI showed a spinal cord conus mass which was biopsied and found to be anaplastic astrocytoma. Therapy included field radiation t10-l1 followed by 11 cycles of temozolomide 7 days on and 7 days off. This was followed by CPT-11 Weekly x4 with Avastin Q2 weeks/ 2 weeks rest and repeat cycle. \n'

In [ ]:
stop_words = ['1-2', '1-2 times', '1 time per day', '2 times', '3 mm', '3 times', '4 times', '5 mm', 
              '6-10 times',
              'abdominal', 'Abreaction', 'absences', 'Activity', 'admission', 'advice', 'anesthesia', 
              'anesthesias', 'Anesthetic', 
              'anesthetic', 'Appearance', 'Application', 'application', 'applications', 'arm',
              'Asleep', 'asleep', 'asymptomatic', 'at admission', 'At home', 'attacks',
              'Bacterium', 'bleed', 'bleeding', 'bleeding time', 'Blood', 'blood', 'Bloody', 'burning',
              'burnings',
              'carried', 'Chronic', 'Center', 'Clearance', 'clinical diagnosis', 'Coagulation', 'coagulation',
              'complaints', 'condition', 'confirmed', 'consideration', 'Constriction', 'constriction', 
              'Consultation', 'consultation', 'consultations', 'Contact', 'contact',
              'day', 'decreas', 'decreased', 'determination', 'Development', 'development', 'diagnose', 
              'diagnosed', 'diagnosis', 'difficulty', 'disc', 'disappointment', 'Discharge',
              'discharge', 'Discharged', 'disease', 'diseases', 'disorders', 'drugs', 'dysfunction',
              'effect', 'elevation', 'emergency', 'Emotional', 'emotional', 'Erythrocytes', 'Examination', 
              'examination', 'examinations', 'Examined', 'examined', 'Exercise', 'exercises', 'extraction',
              'falls', 'field', 'first sign', 'first signs', 'Followed', 'follow-up', 'food',
              'gas', 'General', 'grade', 'gradex', 'groups',
              'Hemoglobin', 'hospitalized',
              'Improved', 'improved', 'Increased', 'injection', 'intervention', 'intolerance', 'Intolerance', 
              'irradiation', 'irritable',
              'Laboratory', 'lead increased', 'lesion', 'lesions', 'Leukocytes', 'Local', 'Low', 'low',
              'Medical', 'Medications', 'medications', 'Monitoring', 'monitoring', 'Multifocals',
              'Negative', 'negative', 'No development', 'noises', 'Nonspecific',
              'organ', 'organs', 'outpatient treatment',
              'Pain', 'pain', 'pains', 'palpation', 'pathology', 'Perform', 'Physical', 'place of residence',
              'plication', 'Positive', 'positive', 'Potassium+', 'prescribed', 'Presence', 'presence',
              'presences', 'Present', 'present', 'preserved', 'Pressure', 'pressure', 'pressures', 
              'Prevention', 'prevention', 'Prophylactic', 'Prosthesis', 'prosthesis', 'Protein', 'psych',
              'Reapplication', 'red blood cells', 'Referred', 'Reirradiation', 'Regional', 'Registered', 
              'Release', 'release', 'removal', 'Repair', 'repair', 'repairs', 'report',  'Report', 'reports',
              'resistance index', 'Result',
              'seizures', 'setting', 'Severe', 'severe', 'sharp pain', 'shortening', 'Single', 'single', 
              'Smoking', 'spells', 
              'stabilize', 'stage', 'Surgical', 'stabilized', 'study', 'surgery', 'surgical intervention', 
              'Susceptible', 'swollen', 'Symptomatic', 'syndrome',
              'Therapeutic', 'Therapy', 'therapy', 'Thoracic', 'thr', 'Thrombocytes', 'Tolerance', 'tolerance', 
              'tolerances', 'Treatment', 'treatment',
              'ultrasound', 'ultrasounds', 'Undifferentiated',
              'W exercise', 'weakness', 'weight',
              'year']

In [ ]:
#labels = ['conditions', 'disease_attrs', 'procedures', 'drugs'] #ver 1
#labels = ['conditions', 'disease_attrs', 'procedures', 'drugs'] #+ attr ver 2
labels = ['conditions', 'disease_attrs', 'procedures', 'drugs', 'systemic_treatments'] #ver 3 - best option
#labels = ['conditions', 'disease_attrs', 'procedures', 'drugs', 'systemic_treatments', 'measurements_and_observations'] #ver 4
ner_for_texts = {}
for i in ner_data.keys():
  i_text_ents = []
  attr = None
  for j in ner_data[i]:
    for ann in j[1]:
      for res in ann['result']:
        try:
          if res['value']['labels'][0] in labels:
            entity = res['value']['text'].lower()
            if len(entity) > 3 and entity not in stop_words and entity not in i_text_ents:
              i_text_ents.append(entity)
            if attr != None:
              #print(res)
              entity = attr + ' ' + entity
              attr = None
              #print(entity)
              i_text_ents.append(entity)
            if res['value']['labels'][0] == 'disease_attrs':
              #print(res)
              attr = entity
              #print(attr)
        except:
          continue
  ner_for_texts[i] = i_text_ents

In [ ]:
with open('true_ner_results_4.json', 'w') as f:
  json.dump(ner_for_texts, f)